In [618]:
import pandas as pd
import numpy as np

In [619]:
oshima = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Dataset_Processing/yunyoung/data/translate.csv', index_col=0)
display(oshima.head())

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살


In [620]:
#pd.options.display.max_rows = 9999
oshima['info'].value_counts().head(10)

火災による死亡            317
死体発見               238
飛び降り自殺             168
自殺                 142
孤高死                101
告知事項あり（不動産情報より）     94
首吊り自殺               77
転落死                 71
刺殺                  64
告知事項あり              60
Name: info, dtype: int64

In [621]:

#TODO : oshima 범죄 카테고리 상위 10개 
#TODO : < 1차적 목표 > 화재, 시체발견, 자살, 투신자살, 추락사, 부동산 고지사항 카테고리 범주 안에서 태깅 목표(완료)
#TODO : < 2차적 목표 > 상위 10개 제외한 예외 경우 새로운 카테고리 만들어서 태깅
#TODO : < 3차적 목표 > 고지사항의 경우 특정 죽음 키워드(사망, 자살)이 포함된 안내사항 인 경우 태깅하고 그렇지 않으면 데이터 삭제

#? : 告知事項あり（不動産情報より）와 告知事項あり는 같은 부동산 고지사항인데 분류된 기준이 뭐일까? -> 부동산 고지사항에 해당하는 부분 좀 더 파보자.
#* 火災による死亡           317 -> 火災 : 화재
#* 死体発見               238 -> 시체 발견 -> 死体 : 시체 
#* 飛び降り自殺             168 -> 높은 곳에서 뛰어 내려 자살(투신자살) -> 自殺 : 자살, 추가 info
#* 自殺                 142 -> 자살 
#* 孤高死                101 -> 고독사
#* 告知事項あり（不動産情報より）     94 -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용, 좀 더 강조된 걸까?
#* 首吊り自殺               77 -> 목 매달아 자살 -> 자살, 추가 info
#* 転落死                 71 -> 낙하사망(추락사)
#* 刺殺                  64 -> 찔러서 죽인 행위 -> 타살
#* 告知事項あり            60 -> -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용 
#! ['火災', '死体', '自殺', '孤高死', '告知事項あり', '転落死', '刺殺'] -> 화재, 시체, 자살, 고독사, 고지사항, 추락사, 타살

In [622]:
oshima.shape

(4833, 8)

In [623]:
#* 주소 중복 데이터 확인됨
df_unique = oshima.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\r\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\r\n특별 모집 주택


In [624]:
# categoryList = oshima['info'].value_counts().head(10).index.tolist()
# categoryList

In [625]:
#* 상위 10개에 해당하는 데이터와 아닌 데이터로 구분
# oshimaTop10 = oshima[oshima['info'].isin(categoryList)].reset_index(drop=True)
# oshimaNotTop10 = oshima[~oshima['info'].isin(categoryList)].reset_index(drop=True)
# print(oshimaTop10.shape, oshimaNotTop10.shape)

In [626]:
# oshimaTop10.loc[:, 'crimeType'] = np.nan
# oshimaNotTop10.loc[:, 'crimeType'] = np.nan

In [627]:
#* 카테고리 7개로 축소
#! ['火災', '死体', '自殺', '孤高死', '告知事項あり', '転落死', '刺殺'] -> 화재, 시체, 자살, 고독사, 고지사항, 추락사, 타살
# keyword_mapping = {
#     '火災': '화재',
#     '死体': '시체',
#     '自殺': '자살',
#     '孤高死': '고독사',
#     '告知事項あり': '부동산고지',
#     '転落死': '추락사',
#     '刺殺': '타살'
# }

In [628]:
keyword_mapping = {
    
    '火災': '화재',
    '死体': '시체',
    '遺体' : '시체',
    'ミイラ' : '시체',
    '入居者死亡を発見' : '시체',
    '腐敗臭' : '시체',
    '自殺': '자살',
    '首吊り' : '자살',
    '飛び降り' : '자살',
    '無理心中' : '자살',
    '縊死' : '자살',
    '心中' : '자살',
    'から投身' : '자살',
    '孤高死': '고독사',
    '孤独死' : '고독사',
    'て独' : '고독사',
    '餓死' : '고독사',
    '告知事項' : '부동산고지',
    '心理的瑕疵' : '부동산고지',
    '事故物件' : '주택내사망',
    '特別募集' : '주택내사망',
    '住民が死亡' : '주택내사망',
    '事故住宅' : '주택내사망',
    '転落死': '추락사',
    'から転落' : '추락사',
    '転落' : '추락사',
    '墜落死' : '추락사',
    '刺殺': '타살',
    '絞殺' : '타살',
    '射殺' : '타살',
    '車両事故死' : '타살',
    '傷害致死' : '타살',
    '撲殺' : '타살',
    '殺人' : '타살',
    '殴殺' : '타살',
    '虐待' : '타살',
    '傷害のち死亡' : '타살',
    '保護責任者遺棄' : '타살',
    '暴行死' : '타살',
    '病死' : '질병사',
    '過労死' : '질병사',
    '熱中症による死亡' : '질병사',
    '業務上過失致死' : '질병사',
    '自然死' : '질병사',
    'コロナ死' : '질병사',
    '倒れていて死亡' : '질병사',
    '事故死' : '사고사',
    'エレベータと壁に挟まれ死亡' : '사고사',
    '溺死' : '사고사',
    '中毒' : '사고사',
    '死亡事故' : '사고사',
    '患者死亡' : '사고사',
    '水没し亡くなっている' : '사고사',
    '変死' : '의문사',
    '不審死' : '의문사',
    '突然死' : '의문사',  
    '原因不明の事故' : '의문사',
    '不自然死' : '의문사',
    '未解決事件' :'의문사',
    '不明' : '의문사',
    '精神的瑕疵' : '심신미약',
    '特別募集住宅' : '기타'}

In [629]:
df_unique.loc[:, 'crimeType'] = np.nan

for idx in range(df_unique.shape[0]):
    for key, value in keyword_mapping.items():
        try:
            if key in str(df_unique.loc[idx, 'info']):
                df_unique.loc[idx, 'crimeType'] = value
                break
        except KeyError:
            continue

In [630]:
df_crime_na = df_unique.loc[df_unique['crimeType'].isna(), ['info', 'translate', 'crimeType']]
df_crime_na

,info,translate,crimeType
24,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より),호텔 파샤 702호실에서 20~30대 남성이 사망하고 있는 것을 발견. (관보에서),NaN
25,2012.飛降り,2012. 비강,NaN
33,不動産業者より告知物件との報告有,부동산업자로부터 고지물건과의 보고유,NaN
49,告知事项あり,고지사 项 있음,NaN
51,2階休息コーナー 脳内出血による死亡,2층 휴식 코너 뇌내 출혈에 의한 사망,NaN
...,...,...,...
3513,瑕疵物件,하자 부동산,NaN
3579,幼児児童を突き落として殺害,유아아동을 찌르고 살해,NaN
3581,幼児を突き落として殺害,유아를 찌르고 살해,NaN
3590,溺殺,익사,NaN


In [631]:
df_crime_na['info'].value_counts().head(20)

トイレで心肺停止女性発見、その後死亡                                                                                                2
ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より)                                                                          1
工事車両が、通行人をひき死亡する。                                                                                                 1
パンチン景品交換所店員が強殺。                                                                                                   1
窒息させて殺害                                                                                                           1
まずこのアパートの住所が違います。\r\nここにアパートはありません。\r\n井口小学校の前の物件です。\r\n\r\nしかも、アパートの方と話しましたがそんなことはないと言っていました。\r\n近所なので営業妨害です。    1
ラブホテルの1号室と17号室でカップル同士の喧嘩で起きた殺傷事件が発生した場所...現在心霊スポットとしてされていて物件とも販売されている場所であるが依然として買い手は、見つからない                       1
独居の家主(男性)がトイレで死亡。数日後別居の長女が発見。                                                                                     1
水道工事中、地下約３メートルのところで男性従業員(29)が生き埋めとなり死亡。                                 

In [632]:
df_crime_na.isnull().sum()

info           0
translate      0
crimeType    241
dtype: int64

In [633]:
df_crime_na = df_unique.loc[df_unique['crimeType'].isna(), ['info', 'translate', 'crimeType']]
df_crime_na

,info,translate,crimeType
24,ホテルパシャ702号室にて20～30代の男性が死亡しているのを発見。(官報より),호텔 파샤 702호실에서 20~30대 남성이 사망하고 있는 것을 발견. (관보에서),NaN
25,2012.飛降り,2012. 비강,NaN
33,不動産業者より告知物件との報告有,부동산업자로부터 고지물건과의 보고유,NaN
49,告知事项あり,고지사 项 있음,NaN
51,2階休息コーナー 脳内出血による死亡,2층 휴식 코너 뇌내 출혈에 의한 사망,NaN
...,...,...,...
3513,瑕疵物件,하자 부동산,NaN
3579,幼児児童を突き落として殺害,유아아동을 찌르고 살해,NaN
3581,幼児を突き落として殺害,유아를 찌르고 살해,NaN
3590,溺殺,익사,NaN


In [634]:
df_crime_na.to_csv('tagging_csv', index=False)